In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
import pandas as pd
import numpy as np

## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose0 = self.datas[0].close
        self.dataclose1 = self.datas[1].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)
        
        #Add an ExponentialMovingAverage indicator
        self.ema = bt.indicators.ExponentialMovingAverage(self.datas[0],period=self.params.maperiod)
        
        #Add Fibonacci pivot point
        self.pivotindicator = bt.indicators.FibonacciPivotPoint(self.datas[1])
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f, SMA, %.2f, EMA, %.2f, support2, %.2f, resistance2, %.2f, position, %.2f' % 
                 (self.dataclose0[0], self.sma[0], self.ema[0], self.pivotindicator.s2[0],self.pivotindicator.r2[0],
                 self.position.size))

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if ...
            #trying reversal strategy
            if (((self.dataclose0[0] < self.ema[0]) & (self.dataclose0[0] < self.sma[0])) &
                (self.dataclose0[0]<self.pivotindicator.s1[0])): 
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose0[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:
            
            if (((self.dataclose0[0] > self.ema[0]) & (self.dataclose0[0] > self.sma[0])) &
                (self.dataclose0[0]>self.pivotindicator.r1[0])):
                                           
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose0[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('(MA Period %2d) Ending Value %.2f' %
                 (self.params.maperiod, self.broker.getvalue()), doprint=True)

## Strategy Plot

In [ ]:
class FixedSlippageAndCommisionScheme(bt.CommInfoBase):
    '''Use this for both slippage and commissions together'''
    params = (
        ('commission', 10), #change slippage to 10
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_FIXED),
        )

    def _getcommission(self, size, price, pseudoexec):
        return self.p.commission

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))

df1 = (df.resample('1h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df1.columns = [col_name.lower() for col_name in df.columns]
df1 = df1.dropna()

df2 = (df.resample('1d', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df2.columns = [col_name.lower() for col_name in df.columns]
df2 = df2.dropna()


In [ ]:
print(df1.head(5))
print(df2.head(5))

print(df1.tail(5))
print(df2.tail(5))

In [ ]:

data1 = bt.feeds.PandasData(dataname = df1['2016-01-01':'2018-01-01'])
data2 = bt.feeds.PandasData(dataname = df2['2016-01-01':'2018-01-01'])

# Add the Data Feed to Cerebro
cerebro.adddata(data1)
cerebro.adddata(data2)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

cerebro.addsizer(bt.sizers.FixedSize, stake=100)

cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Alternatively, you can use:
# cerebro.broker.setcommission(commission=0.0)
# cerebro.broker.set_slippage_fixed(.00,  slip_open=True, slip_limit=True, slip_match=True, slip_out=True)


# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Sharpe Ratio: %.2f' % result[0].analyzers.sharpe_ratio.get_analysis()['sharperatio'])
print('Total Return: %.2f' % result[0].analyzers.returns.get_analysis()['rtot'])
print('Total Trades: %.0f' % result[0].analyzers.trade_analyzer.get_analysis()['total']['total'])
cerebro.plot(volume=False, iplot=False)

In [ ]:
print(result[0].analyzers.trade_analyzer.get_analysis()['long'])

In [ ]:
figure = cerebro.plot()[0][0]
figure.savefig('quant_level4.png')

## In-sample Optimization

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(
    TestStrategy,
    maperiod=range(10, 31),
    printlog=False)

# Add the Data Feed to Cerebro
cerebro.adddata(data1)
cerebro.adddata(data2)

# Set our desired cash start
cerebro.broker.setcash(100000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Run over everything
results = cerebro.run(maxcpus=1)



In [ ]:
ma_period=range(10,31)
sharpe_ratio = [results[i][0].analyzers.sharpe_ratio.get_analysis()['sharperatio'] for i,_ in enumerate(results)]
returns = [results[i][0].analyzers.returns.get_analysis()['rtot'] for i,_ in enumerate(results)]
total_trades = [results[i][0].analyzers.trade_analyzer.get_analysis()['total']['total'] for i,_ in enumerate(results)]
sma_stat= list(zip(ma_period, sharpe_ratio, returns, total_trades))

In [ ]:
for sma_stat_i in sma_stat: print(sma_stat_i)

In [ ]:
max_sharpe_strat= ma_period[np.argmax(sharpe_ratio)]
max_return_strat = ma_period[np.argmax(returns)]
print(max_sharpe_strat, max_return_strat)

In [ ]:
#highest sharpe is 11 but highest return is 13
#trade slightly below what we would like which is 150 trades
#probably very overfit as well but just toy strategy for now

## Out-of-Sample

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TestStrategy, maperiod=max_sharpe_strat)


data = bt.feeds.PandasData(dataname = df['2018-01-01':'2018-06-01'])


# Add the Data Feed to Cerebro
cerebro.adddata(data1)
cerebro.adddata(data2)

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio', timeframe = bt.TimeFrame.Months)
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

# We're trading futures, so we'll use the "commissions" to handle both slippage and commissions
slippage_and_comms = FixedSlippageAndCommisionScheme()
cerebro.broker.addcommissioninfo(slippage_and_comms)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
result_oos = cerebro.run()

# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Total return: %.4f' % result_oos[0].analyzers.returns.get_analysis()['rtot'] )
print('Sharpe Ratio: %.2f' % result_oos[0].analyzers.sharpe_ratio.get_analysis()['sharperatio'] )
print('Total trades: %.0f' % result_oos[0].analyzers.trade_analyzer.get_analysis()['total']['total']  )
# cerebro.plot(volume=False, iplot=False)